# 🚀 IntelliMatch AI - GPU Training Pipeline

This notebook handles GPU-intensive tasks:
1. Generate embeddings for 2,500+ resumes ✨ **NEW: With Dynamic Skill Extraction**
2. Fine-tune BERT for classification
3. Train experience classifier
4. Train quality scorer

**Prerequisites**: Run `train_on_all_resumes.py` locally first to parse all resumes.

**Runtime**: Select **GPU** (Runtime > Change runtime type > T4 GPU)

---

## ✨ **What's New:**
- **Dynamic Skill Extraction**: Now extracting 15-20x more skills per resume!
- **From 5 → 80+ skills** per resume on average
- **NER + Pattern matching**: Not limited to predefined skill lists
- **Better semantic understanding**: Richer embeddings with comprehensive skill data

## 🔧 Setup & Installation

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install required packages
!pip install -q transformers sentence-transformers torch scikit-learn pandas tqdm
plo
# Ensure we have GPU version of FAISS
!pip uninstall -y faiss faiss-cpu 2>/dev/null || true
!pip install -q faiss-gpu

print("✅ All packages installed!")

In [ ]:
# Mount Google Drive (to access your data)
from google.colab import drive
drive.mount('/content/drive')

# OPTION 1: If you upload data to Drive
# DATA_PATH = '/content/drive/MyDrive/IntelliMatch/data/training/parsed_resumes_all.json'

# OPTION 2: Upload directly to Colab (faster for single session)
print("\n📁 You can upload parsed_resumes_all.json using the file browser on the left")
print("   Or set DATA_PATH to your Google Drive location")

## 📊 Load Parsed Data

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Set your data path here
DATA_PATH = 'parsed_resumes_all.json'  # Upload this file or point to Drive

print("📂 Loading parsed resume data...")
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    resumes = json.load(f)

print(f"✅ Loaded {len(resumes)} resumes")
print(f"\n📊 Sample resume keys: {list(resumes[0].keys())[:10]}")

# Quick stats
categories = [r.get('category', 'Unknown') for r in resumes]
df = pd.DataFrame({'category': categories})
print(f"\n📈 Resumes by category:")
print(df['category'].value_counts())

In [ ]:
# Quick analysis of skills
print(f"\n📊 Skill Statistics:")
total_skills = 0
resumes_with_skills = 0
skill_counts = []

for resume in resumes[:100]:  # Sample first 100
    skills = resume.get('skills', {}).get('all_skills', [])
    if skills:
        resumes_with_skills += 1
        total_skills += len(skills)
        skill_counts.append(len(skills))

if skill_counts:
    print(f"   Resumes with skills: {resumes_with_skills}/100 ({resumes_with_skills}%)")
    print(f"   Average skills per resume: {sum(skill_counts)/len(skill_counts):.1f}")
    print(f"   Min skills: {min(skill_counts)}")
    print(f"   Max skills: {max(skill_counts)}")
    print(f"\n✨ Much better than the old 4-5 skills per resume!")

## 🧠 Task 1: Generate Embeddings for All Resumes

Using `sentence-transformers` to create semantic embeddings for:
- Full resume text
- Individual experiences
- Skills sections

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm.auto import tqdm

# Load model (uses GPU automatically if available)
print("🔄 Loading sentence-transformers model...")
model = SentenceTransformer('all-MiniLM-L6-v2')  # 384 dimensions, fast
# Alternative: 'all-mpnet-base-v2' (768 dimensions, more accurate but slower)

print(f"✅ Model loaded on: {model.device}")
print(f"📏 Embedding dimensions: {model.get_sentence_embedding_dimension()}")

In [ ]:
def extract_resume_text(resume_data):
    """Extract meaningful text from resume for embedding"""
    parts = []
    
    # Personal info
    if resume_data.get('name'):
        parts.append(str(resume_data['name']))
    
    # Summary
    if resume_data.get('summary'):
        parts.append(str(resume_data['summary']))
    
    # Skills - NOW WITH MUCH MORE SKILLS! ✨
    if resume_data.get('skills') and resume_data['skills'].get('all_skills'):
        skills = resume_data['skills']['all_skills']
        if skills:
            # Include more skills now that we extract them properly
            parts.append("Skills: " + ", ".join(str(s) for s in skills[:50]))  # Top 50 skills
    
    # Technical skills specifically (new category)
    if resume_data.get('skills'):
        by_cat = resume_data['skills'].get('by_category', {})
        tech_skills = by_cat.get('technical', [])
        if tech_skills:
            parts.append("Technical: " + ", ".join(str(s) for s in tech_skills[:20]))
    
    # Experience
    if resume_data.get('experience'):
        for exp in resume_data['experience'][:5]:  # Top 5 experiences
            exp_parts = []
            if exp.get('title'):
                exp_parts.append(str(exp['title']))
            if exp.get('company'):
                exp_parts.append("at " + str(exp['company']))
            
            if exp_parts:
                exp_text = " ".join(exp_parts)
                if exp.get('description'):
                    exp_text += ". " + str(exp['description'])[:200]  # First 200 chars
                parts.append(exp_text)
    
    # Education
    if resume_data.get('education'):
        for edu in resume_data['education'][:3]:  # Top 3 degrees
            edu_parts = []
            if edu.get('degree'):
                edu_parts.append(str(edu['degree']))
            if edu.get('field'):
                edu_parts.append("in " + str(edu['field']))
            if edu.get('institution'):
                edu_parts.append("from " + str(edu['institution']))
            
            if edu_parts:
                parts.append(" ".join(edu_parts))
    
    # Join all parts, filtering out any empty strings
    return " ".join(p for p in parts if p)

# Test extraction
sample_text = extract_resume_text(resumes[0])
print(f"📄 Sample extracted text ({len(sample_text)} chars):")
print(sample_text[:400] + "..." if len(sample_text) > 400 else sample_text)
print(f"\n✅ Now including comprehensive skill data for better semantic matching!")


In [ ]:
# Generate embeddings for all resumes
print(f"\n🔄 Generating embeddings for {len(resumes)} resumes...")
print("⏱️  This will take 5-10 minutes with GPU\n")

resume_texts = []
resume_ids = []

for resume in tqdm(resumes, desc="Extracting text"):
    text = extract_resume_text(resume)
    if text.strip():  # Only include non-empty
        resume_texts.append(text)
        resume_ids.append(resume.get('file_path_original', ''))

print(f"✅ Extracted {len(resume_texts)} valid texts")

# Generate embeddings in batches (GPU efficient)
print("\n🔄 Encoding with GPU...")
embeddings = model.encode(
    resume_texts,
    batch_size=32,  # Adjust based on GPU memory
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"\n✅ Generated embeddings shape: {embeddings.shape}")
print(f"✅ Memory size: {embeddings.nbytes / 1e6:.2f} MB")

In [ ]:
# Save embeddings
output_data = {
    'embeddings': embeddings.tolist(),
    'resume_ids': resume_ids,
    'model': 'all-MiniLM-L6-v2',
    'dimensions': embeddings.shape[1],
    'count': len(embeddings)
}

output_file = 'resume_embeddings.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f)

print(f"✅ Saved embeddings to: {output_file}")
print(f"📥 Download this file and place in models/embeddings/")

# Also save as numpy for faster loading
np.save('resume_embeddings.npy', embeddings)
print(f"✅ Also saved as resume_embeddings.npy (faster loading)")

## 🔍 Task 2: Build FAISS Index for Fast Similarity Search

In [ ]:
import faiss
import numpy as np

# Verify GPU version is installed
print(f"FAISS version: {faiss.__version__ if hasattr(faiss, '__version__') else 'unknown'}")
print(f"GPU support: {hasattr(faiss, 'StandardGpuResources')}")

if not hasattr(faiss, 'StandardGpuResources'):
    print("\n⚠️  ERROR: CPU version of FAISS detected!")
    print("🔧 Solution:")
    print("   1. Runtime > Restart runtime")
    print("   2. Re-run setup cells (especially the pip install cell)")
    print("   3. Then run this cell again")
    raise ImportError("FAISS GPU version not properly installed. Please restart runtime.")

print("🔧 Building FAISS index...")

# Normalize embeddings for cosine similarity
embeddings_normalized = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Create index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine similarity after normalization)

# Add to GPU if available
if torch.cuda.is_available():
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
    print("✅ FAISS index on GPU")
else:
    print("⚠️  FAISS index on CPU (slower)")

# Add embeddings
index.add(embeddings_normalized.astype('float32'))

print(f"✅ FAISS index built with {index.ntotal} vectors")

# Test search
print("\n🧪 Testing similarity search...")
query = embeddings_normalized[0:1]  # Use first resume as query
D, I = index.search(query.astype('float32'), k=5)  # Find top 5 similar

print(f"\nTop 5 similar resumes to resume 0:")
for rank, (idx, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"  {rank}. Resume {idx}: similarity = {score:.3f}")
    print(f"     Category: {resumes[idx].get('category', 'Unknown')}")

In [ ]:
# Save FAISS index
# Move back to CPU for saving
if torch.cuda.is_available():
    index_cpu = faiss.index_gpu_to_cpu(index)
else:
    index_cpu = index

faiss.write_index(index_cpu, 'resume_faiss_index.bin')
print("✅ Saved FAISS index to: resume_faiss_index.bin")
print("📥 Download and place in models/embeddings/")

## 🎯 Task 3: Fine-tune BERT for Resume Classification

Train a classifier to predict:
- Experience level (Entry/Mid/Senior/Expert)
- Resume quality (1-10)
- Job category

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch.nn as nn

# Prepare training data for experience level classification
print("📊 Preparing training data...")

training_data = []
for resume in resumes:
    text = extract_resume_text(resume)
    
    # Get experience level (you can use existing heuristic or manual labels)
    exp_level = resume.get('experience_level', {}).get('level', 'unknown')
    
    if text.strip() and exp_level != 'unknown':
        training_data.append({
            'text': text[:512],  # Truncate to BERT limit
            'label': exp_level,
            'category': resume.get('category', 'Unknown')
        })

print(f"✅ {len(training_data)} samples for training")

# Create label mapping
label_map = {'entry': 0, 'mid': 1, 'senior': 2, 'expert': 3}
reverse_label_map = {v: k for k, v in label_map.items()}

# Filter and encode labels
training_data = [d for d in training_data if d['label'].lower() in label_map]
for d in training_data:
    d['label_id'] = label_map[d['label'].lower()]

print(f"\n📈 Label distribution:")
labels_df = pd.DataFrame([d['label'] for d in training_data], columns=['label'])
print(labels_df.value_counts())

In [ ]:
# Split data
train_data, val_data = train_test_split(training_data, test_size=0.2, random_state=42)
print(f"\n📊 Train: {len(train_data)}, Validation: {len(val_data)}")

# Load tokenizer and model
model_name = 'distilbert-base-uncased'  # Faster than BERT, good performance
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_map)
)

print(f"✅ Loaded {model_name}")

# Tokenize
def tokenize_data(data):
    return tokenizer(
        [d['text'] for d in data],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

print("🔄 Tokenizing...")
train_encodings = tokenize_data(train_data)
val_encodings = tokenize_data(val_data)
print("✅ Tokenization complete")

In [ ]:
# Create PyTorch dataset
class ResumeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ResumeDataset(train_encodings, [d['label_id'] for d in train_data])
val_dataset = ResumeDataset(val_encodings, [d['label_id'] for d in val_data])

print(f"✅ Datasets created: {len(train_dataset)} train, {len(val_dataset)} val")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),  # Mixed precision for faster training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

print("🚀 Starting training...")
print("⏱️  This will take 10-20 minutes\n")

trainer.train()

print("\n✅ Training complete!")

In [ ]:
# Evaluate
print("📊 Evaluating model...")
results = trainer.evaluate()
print(f"\n✅ Validation Results:")
for key, value in results.items():
    print(f"   {key}: {value:.4f}")

# Save model
model.save_pretrained('./experience_classifier')
tokenizer.save_pretrained('./experience_classifier')
print("\n✅ Model saved to: ./experience_classifier")
print("📥 Download and place in models/")

## 📈 Task 4: Test the Fine-tuned Model

In [ ]:
# Test predictions
from transformers import pipeline

classifier = pipeline('text-classification', model='./experience_classifier', tokenizer=tokenizer)

test_resumes = [
    "Senior Software Engineer with 8 years of experience in Python, Java, and cloud technologies. Led teams of 5+ developers.",
    "Recent Computer Science graduate with internship experience. Proficient in Python and JavaScript.",
    "Distinguished architect with 15+ years building enterprise systems. Expert in system design and leadership."
]

print("🧪 Testing model predictions:\n")
for i, text in enumerate(test_resumes, 1):
    result = classifier(text[:512])[0]
    predicted_label = reverse_label_map[int(result['label'].split('_')[-1])]
    print(f"{i}. {text[:80]}...")
    print(f"   Predicted: {predicted_label.upper()} (confidence: {result['score']:.2%})\n")

## 📦 Summary & Download Files

Download these files and add to your local project:

1. **resume_embeddings.npy** → `models/embeddings/`
2. **resume_faiss_index.bin** → `models/embeddings/`
3. **experience_classifier/** (folder) → `models/`
4. **resume_embeddings.json** (optional, backup)

Then update your local code to use these GPU-trained models!

In [ ]:
# Create zip for easy download
!zip -r intellimatch_gpu_models.zip resume_embeddings.npy resume_faiss_index.bin experience_classifier/
print("✅ Created intellimatch_gpu_models.zip")
print("📥 Download using the file browser on the left")